In [1]:
!python -m spacy download nb_core_news_sm
!python -m spacy download nb_core_news_md
!python -m spacy download nb_core_news_lg
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 12.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('nb_core_news_sm')
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 11.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('nb_core_news_md')
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.4/569.4 MB 3.8 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('nb_core_news_lg')
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import spacy
import json
import pandas
from numpy import arange
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

In [3]:
data_3class = {}
for name in ["train", "dev", "test"]:
    with open(f"norec_sentence/3class/{name}.json") as infile:
        data_3class[name] = json.load(infile)


The data from thre json files named train, dev, and test are loaded are collected in the two dictionaries data_3class and data_binary

Dataset cloned from github: https://github.com/ltgoslo/norec_sentence

Kutuzov, A., Barnes, J., Velldal, E., Øvrelid, L., & Oepen, S. (2021). Large-Scale Contextualised Language Modelling for Norwegian. Proceedings of the 23rd Nordic Conference on Computational Linguistics (NoDaLiDa 2021).

Øvrelid, L., Mæhlum, P., Barnes, J., & Velldal, E. (2020). A Fine-grained Sentiment Dataset for Norwegian. Proceedings of the 12th Edition of the Language Resources and Evaluation Conference. Marseille, France, 2020.

The NoReC dataset is a collection of data from reviews, since it comes from reviews it can easely be maped to sentiment, ass most reviews come with a rating.

In [4]:
data_3class['train']

[{'sent_id': '201911-01-01', 'text': 'Philips 190G6', 'label': 'Neutral'},
 {'sent_id': '201911-02-01',
  'text': 'Med integrerte høyttalere som på ingen måte er diskret plassert , og med en stor subwoofer inkludert , da snakker vi om en gutteskjerm .',
  'label': 'Neutral'},
 {'sent_id': '201911-02-02',
  'text': 'Eller bedrar skinnet ?',
  'label': 'Negative'},
 {'sent_id': '201911-03-01',
  'text': 'De fleste skjermer har et diskret design , med smale rammer og slank fot .',
  'label': 'Neutral'},
 {'sent_id': '201911-03-02',
  'text': 'Men 190G6 fra Philips er en helt annen historie .',
  'label': 'Neutral'},
 {'sent_id': '201911-03-03',
  'text': 'Den har et utseende som krever oppmerksomhet , med glinsende svart ramme , glansbelegg på skjermflaten og store sølvfargede sidepaneler med fire innfelte høyttalere med svart deksel .',
  'label': 'Neutral'},
 {'sent_id': '201911-04-01', 'text': 'LES OGSÅ :', 'label': 'Neutral'},
 {'sent_id': '201911-05-01',
  'text': 'Foten har en stor 

In [5]:
for sent in data_3class['train']:
    print(sent['text'])

Philips 190G6
Med integrerte høyttalere som på ingen måte er diskret plassert , og med en stor subwoofer inkludert , da snakker vi om en gutteskjerm .
Eller bedrar skinnet ?
De fleste skjermer har et diskret design , med smale rammer og slank fot .
Men 190G6 fra Philips er en helt annen historie .
Den har et utseende som krever oppmerksomhet , med glinsende svart ramme , glansbelegg på skjermflaten og store sølvfargede sidepaneler med fire innfelte høyttalere med svart deksel .
LES OGSÅ :
Foten har en stor og blank søyle , og det er store knapper og blå lys .
Baksiden er sort , blank og skinnende , med et deksel som skjuler kontakter og kabler .
De fire høyttalerbrønnene stikker tydelig ut - her er det ikke snakk om å gjemme noe .
Likegyldig er det uansett vanskelig å være .
God betjening
I midten finner vi volumknappen , som roterer fritt .
Nivået leser du av på skjermen , det dukker opp en skala så snart du skrur på knappen .
Til venstre finner vi en inngangsvelger , som bestemmer om

From these last three test we see that oure data is orginized in lists of dictionaries, so the "data" variable is a dictionary of lists of dictionaries.

Using bag of words to orginize method to orginize the text into easaly computable vectors

In [6]:
text_3class = {'train':[x['text'] for x in data_3class['train']], 
               'test':[x['text'] for x in data_3class['test']]}

labels_3class ={'train':[x['label'] for x in data_3class['train']],
                'test':[x['label'] for x in data_3class['test']]}

as the original data comes in dictionaries of lists of reviews which are also dictionaries, we chose to re structure the data for our own purposes

In [7]:
print(labels_3class['train'])

['Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Negative', 'Neutral', 'Negative', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Positive', 'Negative', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Negative', 'Positive', 'Negative', 'Neutral', 'Positive', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Neutral'

Notice there are a lot of neutral values

In [21]:
def lemmatizer(list_sentc:list, nlp:spacy, stop:list)->list:
    """takes a list of sentences and returns a list of lemmatized sentences

    Args:
        list_sentc (list): sentences

    Returns:
        list: lemmatized sentences
    """
    result = []
    for sentence in list_sentc:
        doc = nlp(sentence.lower())
        result.append(' '.join([token.lemma_ for token in doc if token.lemma_ not in stop]))

    return result

In [9]:
nb_nlp = {'smal': spacy.load('nb_core_news_sm'), 'medium': spacy.load('nb_core_news_md'), 'large':spacy.load('nb_core_news_lg')}

lemmatizer is reducing the complexity of the text by turning words into their base form also known as lemma, here i test 3 different sizes of norwegian laguage models to lemmatize, later i wil see the difference in accuracy between them.

In [12]:
file_1 = open('stopwords-no.json')
file_2 = open('stopwords_nb.txt')
no_stop = {'short': file_2.read().split('\n'), 'long':json.load(file_1)}

print(no_stop['short'][:50], no_stop['long'][:50])

['alle', 'andre', 'arbeid', 'av', 'begge', 'bort', 'bra', 'bruke', 'da', 'denne', 'der', 'deres', 'det', 'din', 'disse', 'du', 'eller', 'en', 'ene', 'eneste', 'enhver', 'enn', 'er', 'et', 'folk', 'for', 'fordi', 'forsÛke', 'fra', 'fÅ', 'fÛr', 'fÛrst', 'gjorde', 'gjÛre', 'god', 'gÅ', 'ha', 'hadde', 'han', 'hans', 'hennes', 'her', 'hva', 'hvem', 'hver', 'hvilken', 'hvis', 'hvor', 'hvordan', 'hvorfor'] ['alle', 'andre', 'arbeid', 'at', 'av', 'bare', 'begge', 'ble', 'blei', 'bli', 'blir', 'blitt', 'bort', 'bra', 'bruke', 'både', 'båe', 'da', 'de', 'deg', 'dei', 'deim', 'deira', 'deires', 'dem', 'den', 'denne', 'der', 'dere', 'deres', 'det', 'dette', 'di', 'din', 'disse', 'ditt', 'du', 'dykk', 'dykkar', 'då', 'eg', 'ein', 'eit', 'eitt', 'eller', 'elles', 'en', 'ene', 'eneste', 'enhver']


In [13]:
text = text_3class['train']

test_sm = TfidfVectorizer(use_idf=False).fit(lemmatizer(text, nb_nlp['smal']))
test_md = TfidfVectorizer(use_idf=False).fit(lemmatizer(text, nb_nlp['medium']))
test_lg = TfidfVectorizer(use_idf=False).fit(lemmatizer(text, nb_nlp['large']))
compare = TfidfVectorizer(use_idf=False).fit(text)

print('smal norgwegian model pecent remove:', 1-len(test_sm.get_feature_names_out())/len(compare.get_feature_names_out()))
print('medium norgwegian model pecent remove:', 1-len(test_md.get_feature_names_out())/len(compare.get_feature_names_out()))
print('large norgwegian model pecent remove:', 1-len(test_lg.get_feature_names_out())/len(compare.get_feature_names_out()))

smal norgwegian model pecent remove: 0.18522612582304032
medium norgwegian model pecent remove: 0.1933483923679531
large norgwegian model pecent remove: 0.20137453741529288


only removing about 20% of words with the largest model seems a bit low considdering the possible number of forms for nowegian word

In [23]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                ('clf', MultinomialNB())])

parameters = {'clf__alpha': [0.01, 0.1, 1],
              'vect__use_idf': [True, False],
              'vect__ngram_range': [(1,1), (1,2), (1,3)]}

# differing complexity data might require different methods, 
# therefore i use grid search to find the best parameters for each set of data that has been lematized in differente ways
text = text_3class['train'] + text_3class['test']
labels = labels_3class['train'] + labels_3class ['test']

grid = GridSearchCV(pipe, parameters, n_jobs=-1)
grid.fit(text, labels)

print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)
print('Size of model: none')
print('Size of stopword list: none')

for model in nb_nlp:
    grid = GridSearchCV(pipe, parameters, n_jobs=-1)
    grid.fit(text, labels)

    print('Best parameters:', grid.best_params_)
    print('Best score:', grid.best_score_)
    print('Size of model:', model)
    print('Size of stopword list: none')

    for stop in no_stop:
        text = lemmatizer(text, nb_nlp[model], no_stop[stop])

        grid = GridSearchCV(pipe, parameters, n_jobs=-1)
        grid.fit(text, labels)

        print('Best parameters:', grid.best_params_)
        print('Best score:', grid.best_score_)
        print('Size of model:', model)
        print('Size of stopword list:', stop)


Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 1), 'vect__use_idf': False}
Best score: 0.6159053698746243
Size of model: none
Size of stopword list: none
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 1), 'vect__use_idf': False}
Best score: 0.5971150167276981
Size of model: smal
Size of stopword list: short
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.5978803424925314
Size of model: smal
Size of stopword list: long
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.5989722239631364
Size of model: medium
Size of stopword list: short
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.597552055820672
Size of model: medium
Size of stopword list: long
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.599190564444166
Size of model: large
Size of s

output:
```
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 1), 'vect__use_idf': False}
Best score: 0.5971150167276981
Size of model: smal
Size of stopword list: short
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.5978803424925314
Size of model: smal
Size of stopword list: long
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.5989722239631364
Size of model: medium
Size of stopword list: short
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.597552055820672
Size of model: medium
Size of stopword list: long
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.599190564444166
Size of model: large
Size of stopword list: short
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.5995183139196533
Size of model: large
Size of stopword list: long
```
here we se that the large language model with the long stopword list gives us the highest acuracy after preprocessing, word counts are better than word weights for the moddel, and we don't gain performans from ngrams larger than 2.

In [54]:
pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), use_idf=False)),
                ('clf', MultinomialNB())])

alphas = [round(x, 3) for x in arange(0.01, 0.16, 0.01)]
# every number between 0.1 and 0.01 with a difference of 0.01

parameters = {'clf__alpha': alphas}

text = text_3class['train']+text_3class['test']
text = lemmatizer(text, nb_nlp['medium'])

labes = labels_3class['train']+text_3class['test']

grid = GridSearchCV(pipe, parameters, n_jobs=-1)
grid.fit(text, labels)

print(grid.best_score_, grid.best_params_)

0.621476454384567 {'clf__alpha': 0.1}


same best alpha, but the model performes slightly better, this is probably because i did not lemmatize the test data, the better performance is therefore the moddel generelizing better because the training data is different from the test data, we don't actualy want this, because the better performance is not due to a better rather a model that predicts more randomly

In [70]:
pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), use_idf=False)),
                ('clf', MultinomialNB(alpha=0.1))])

text = lemmatizer(text_3class['train'], nb_nlp['medium'])

pipe.fit(text, labels_3class['train'])

prediction = pipe.predict(text_3class['test'])

conf_matrix = confusion_matrix(prediction, labels_3class['test'], labels=['Positive', 'Neutral', 'Negative'])

matrix = pandas.DataFrame(conf_matrix, index= ['Positive', 'Neutral', 'Negative'], columns=['Positive', 'Neutral', 'Negative'])

print(matrix)

          Positive  Neutral  Negative
Positive       131       73        36
Neutral        268      518       137
Negative         2        7         9


rows are the true value, whil the columns are what MultinomialNB predicted. we see that multinomialNB is bad at detecting negative sentiment, it predicts sentiment as neutral when it should be negative, this can be due to using word count in oure vectors instead of word weight.

In [71]:
pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2))),
                ('clf', MultinomialNB(alpha=0.1))])

text = lemmatizer(text_3class['train'], nb_nlp['medium'])

pipe.fit(text, labels_3class['train'])

prediction = pipe.predict(text_3class['test'])

conf_matrix = confusion_matrix(prediction, labels_3class['test'], labels=['Positive', 'Neutral', 'Negative'])

matrix = pandas.DataFrame(conf_matrix, index= ['Positive', 'Neutral', 'Negative'], columns=['Positive', 'Neutral', 'Negative'])

print(matrix)

          Positive  Neutral  Negative
Positive       153       90        40
Neutral        243      496       123
Negative         5       12        19


we see that it get's better at predicting negative and positive sentiment but not neutral. we might be limited by oure classifier. 

In [24]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                ('clf', BernoulliNB())])

parameters = {'clf__alpha': [0.01, 0.001, 0.1, 1],
              'vect__use_idf': [True, False],
              'vect__ngram_range': [(1,1), (1,2), (1,3)]}

text = text_3class['train'] + text_3class['test']
labels = labels_3class['train'] + labels_3class ['test']

print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)
print('Size of model: none')
print('Size of stopword list: none')

for model in nb_nlp:
    for stop in no_stop:
        text = lemmatizer(text, nb_nlp[model], no_stop[stop])

        grid = GridSearchCV(pipe, parameters, n_jobs=-1)
        grid.fit(text, labels)

        print('Best parameters:', grid.best_params_)
        print('Best score:', grid.best_score_)
        print('Size of model:', model)
        print('Size of stopword list:', stop)

Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2), 'vect__use_idf': False}
Best score: 0.5995183139196533
Size of model: none
Size of stopword list: none
Best parameters: {'clf__alpha': 0.1, 'vect__ngram_range': (1, 1), 'vect__use_idf': True}
Best score: 0.5934006022568217
Size of model: smal
Size of stopword list: short


KeyboardInterrupt: 

BernoulliNB performed slightly worse

In [76]:
pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2))),
                ('clf', BernoulliNB(alpha=0.1))])

text = lemmatizer(text_3class['train'], nb_nlp['medium'])

pipe.fit(text, labels_3class['train'])

prediction = pipe.predict(text_3class['test'])

conf_matrix = confusion_matrix(prediction, labels_3class['test'], labels=['Positive', 'Neutral', 'Negative'])

matrix = pandas.DataFrame(conf_matrix, index= ['Positive', 'Neutral', 'Negative'], columns=['Positive', 'Neutral', 'Negative'])

print(matrix)

          Positive  Neutral  Negative
Positive        84       34        20
Neutral        308      557       148
Negative         9        7        14


even less accurate at predicting sentiment, it seems like the neutral values are being predicted falsly as positive or negative.

the two linear models are proably bad at predicting neutral values because they try to draw a plane, and the line is not "curvy" enough, to fix this, let's try reducing oure alpha/normalization

In [78]:
pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2))),
                ('clf', BernoulliNB(alpha=0.000001))])

text = lemmatizer(text_3class['train'], nb_nlp['medium'])

pipe.fit(text, labels_3class['train'])

prediction = pipe.predict(text_3class['test'])

conf_matrix = confusion_matrix(prediction, labels_3class['test'], labels=['Positive', 'Neutral', 'Negative'])

matrix = pandas.DataFrame(conf_matrix, index= ['Positive', 'Neutral', 'Negative'], columns=['Positive', 'Neutral', 'Negative'])

print(matrix)

          Positive  Neutral  Negative
Positive       163      104        64
Neutral        221      461        91
Negative        17       33        27


ass we see with the example of an extremly small alpa/almost no normalization, we classify the positive and negative values better but overall we are less accurate(the predicted sentiment is more spread out), and as explained erlier this is not something we want

Random forest classifier uses decisiton trees, it makes multiple trees that each try to classify the data and then it takes the result of each tree weighted with the probability of the classification to be true and finds the moste probable answere. since it makes decision trees it is not limited by "curvynes" or the fact that there are "islands" of data in the two dimetional field, but this unlimited ability to seperate the field of values also makes it more vonurable to over fitting

In [ ]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                ('clf', RandomForestClassifier(n_jobs=-1, n_estimators=100))])

parameters = {'clf__max_depth': [10, 100, None],
              'vect__use_idf': [True, False],
              'vect__ngram_range': [(1,1), (1,2), (1,3)]}

text = text_3class['train'] + text_3class['test']
labels = labels_3class['train'] + labels_3class ['test']

grid = GridSearchCV(pipe, parameters, n_jobs=-1)
grid.fit(text, labels)

print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)
print('Size of model: none')
print('Size of stopword list: none')

for model in nb_nlp:
    for stop in no_stop:
        text = lemmatizer(text, nb_nlp[model], no_stop[stop])

        grid = GridSearchCV(pipe, parameters, n_jobs=-1)
        grid.fit(text, labels)

        print('Best parameters:', grid.best_params_)
        print('Best score:', grid.best_score_)
        print('Size of model:', model)
        print('Size of stopword list:', stop)

Best parameters: {'clf__max_depth': None, 'vect__ngram_range': (1, 1), 'vect__stop_words': ['alle', 'andre', 'arbeid', 'at', 'av', 'bare', 'begge', 'ble', 'blei', 'bli', 'blir', 'blitt', 'bort', 'bra', 'bruke', 'både', 'båe', 'da', 'de', 'deg', 'dei', 'deim', 'deira', 'deires', 'dem', 'den', 'denne', 'der', 'dere', 'deres', 'det', 'dette', 'di', 'din', 'disse', 'ditt', 'du', 'dykk', 'dykkar', 'då', 'eg', 'ein', 'eit', 'eitt', 'eller', 'elles', 'en', 'ene', 'eneste', 'enhver', 'enn', 'er', 'et', 'ett', 'etter', 'folk', 'for', 'fordi', 'forsûke', 'fra', 'få', 'før', 'fûr', 'fûrst', 'gjorde', 'gjûre', 'god', 'gå', 'ha', 'hadde', 'han', 'hans', 'har', 'hennar', 'henne', 'hennes', 'her', 'hjå', 'ho', 'hoe', 'honom', 'hoss', 'hossen', 'hun', 'hva', 'hvem', 'hver', 'hvilke', 'hvilken', 'hvis', 'hvor', 'hvordan', 'hvorfor', 'i', 'ikke', 'ikkje', 'ingen', 'ingi', 'inkje', 'inn', 'innen', 'inni', 'ja', 'jeg', 'kan', 'kom', 'korleis', 'korso', 'kun', 'kunne', 'kva', 'kvar', 'kvarhelst', 'kven', '

output:
```
Best parameters: {'clf__max_depth': None, 'vect__ngram_range': (1, 1), 'vect__stop_words': no_stop, 'vect__use_idf': False}
Score: 0.6008298490179752
Size of model: smal
Best parameters: {'clf__max_depth': None, 'vect__ngram_range': (1, 1), 'vect__stop_words': no_stop, 'vect__use_idf': False}
Score: 0.6029053370459572
Size of model: medium
Best parameters: {'clf__max_depth': None, 'vect__ngram_range': (1, 1), 'vect__stop_words': no_stop, 'vect__use_idf': True}
Score: 0.6031249906736741
Size of model: large
```

random forest seems to perform best with very preprossesed data, this might be because it generates more "usefull" trees if the data only includes the most usefull words. it also performes better than anny of the linear models, let's try with a larger amount of estimators now that we are not looking for the best parameters.

In [85]:
pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,1), stop_words=no_stop)),
                ('clf', RandomForestClassifier(n_jobs=-1))])

parameters = {'clf__n_estimators': [100, 300, 500, 700, 1000]}

text = text_3class['train']+text_3class['test']
text = lemmatizer(text, nb_nlp['large'])

labels = labels_3class['train'] + labels_3class['test']

grid = GridSearchCV(pipe, parameters, n_jobs=-1)
grid.fit(text, labels)

print('Score:', grid.best_score_, 'Best parms:', grid.best_params_)

Score: 0.6003926905480299 Best parms: {'clf__n_estimators': 1000}


out:
```
Score: 0.6003926905480299 Best parms: {'clf__n_estimators': 1000}
```

In [16]:
pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,1), stop_words=no_stop)),
                ('clf', RandomForestClassifier(n_jobs=-1))])

parameters = {'clf__n_estimators': [1000, 1500, 2000]}

text = text_3class['train']+text_3class['test']
text = lemmatizer(text, nb_nlp['large'])

labels = labels_3class['train'] + labels_3class['test']

grid = GridSearchCV(pipe, parameters, n_jobs=-1)
grid.fit(text, labels)

print('Score:', grid.best_score_, 'Best parms:', grid.best_params_)

Score: 0.6000647023185992 Best parms: {'clf__n_estimators': 1000}


i don't know why we gett lower acuracy, it is probably a quirk of how i programmed the model differently (with the optimal parameters). the best number of estimators seems to be 1000

In [86]:
pipe = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,1), stop_words=no_stop)),
                ('clf', RandomForestClassifier(n_jobs=-1, n_estimators=10000))])

text = lemmatizer(text_3class['train'], nb_nlp['large'])

pipe.fit(text, labels_3class['train'])

prediction = pipe.predict(text_3class['test'])

conf_matrix = confusion_matrix(prediction, labels_3class['test'], labels=['Positive', 'Neutral', 'Negative'])

matrix = pandas.DataFrame(conf_matrix, index= ['Positive', 'Neutral', 'Negative'], columns=['Positive', 'Neutral', 'Negative'])

print(matrix)

          Positive  Neutral  Negative
Positive       101       53        28
Neutral        281      526       124
Negative        19       19        30


again we see that Neutral sentiment get's under predicted, maby oure problem is that the feature space is too sparce.

i therefore try logistic regression, logistic regression creates a statistical moddel that tries to draw a boundery in the feature space, the goal is to classify oure data

In [ ]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                ('clf', LogisticRegression(max_iter=500, n_jobs=-1))])

params = {'clf__C': [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(pipe, params, n_jobs=-1)
grid.fit(text, labels)

score = grid.best_score_
parms = grid.best_params_

print(score, parms)

0.6348028041650624 {'clf__C': 1}


In [ ]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                 ('clf', GradientBoostingClassifier())])

params = {'clf__n_estimators': [100, 200, 300, 400, 500, 600, 700]}

grid = GridSearchCV(pipe, params, n_jobs=-1)
grid.fit(text, labels)

time = grid.cv_results_['mean_fit_time']
scores = grid.cv_results_['mean_test_score']
parameters = grid.cv_results_['params']

for i in range(0, len(time)):
    print(f'time: {time[i]} \tscore: {scores[i]} \tparameters: {parameters[i]}')

time: 9.1149338722229 	score: 0.5989715077013069 	parameters: {'clf__n_estimators': 50}
time: 17.170205211639406 	score: 0.6109887099229123 	parameters: {'clf__n_estimators': 100}
time: 33.38438782691956 	score: 0.6169968932143145 	parameters: {'clf__n_estimators': 200}
time: 53.35387144088745 	score: 0.6192913782966697 	parameters: {'clf__n_estimators': 300}
time: 71.8617947101593 	score: 0.6240979726805801 	parameters: {'clf__n_estimators': 400}
time: 89.77509512901307 	score: 0.6224588074837424 	parameters: {'clf__n_estimators': 500}
time: 105.62572784423828 	score: 0.6274843989220259 	parameters: {'clf__n_estimators': 600}
time: 101.68260288238525 	score: 0.6272656406215958 	parameters: {'clf__n_estimators': 700}


we choose to use Suport vector machines because we have 

In [ ]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                 ('clf', SVC())])

params = {'clf__C': [0.1, 1, 1000],
          'clf__gamma': [0.1, 1, 10],
          'vect__use_idf': [True, False],
          'vect__ngram_range': [(1,1), (1,2), (1,3)]}

text = text_3class['train'] + text_3class['test']
labels = labels_3class['train'] + labels_3class ['test']

grid = GridSearchCV(pipe, parameters, n_jobs=-1)
grid.fit(text, labels)

print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)
print('Size of model: none')
print('Size of stopword list: none')

for model in nb_nlp:
    for stop in no_stop:
        text = lemmatizer(text, nb_nlp[model], no_stop[stop])

        grid = GridSearchCV(pipe, parameters, n_jobs=-1)
        grid.fit(text, labels)

        print('Best parameters:', grid.best_params_)
        print('Best score:', grid.best_score_)
        print('Size of model:', model)
        print('Size of stopword list:', stop)

time: 11.150942420959472 	score: 0.5109241866697705 	parameters: {'clf__C': 0.1, 'clf__gamma': 0.1}
time: 14.382927799224854 	score: 0.5114703363147732 	parameters: {'clf__C': 0.1, 'clf__gamma': 1}
time: 17.20559158325195 	score: 0.5109241866697705 	parameters: {'clf__C': 0.1, 'clf__gamma': 10}
time: 14.470163488388062 	score: 0.5877206459487934 	parameters: {'clf__C': 1, 'clf__gamma': 0.1}
time: 16.74793496131897 	score: 0.6331644746070259 	parameters: {'clf__C': 1, 'clf__gamma': 1}
time: 17.581197452545165 	score: 0.5122350651947486 	parameters: {'clf__C': 1, 'clf__gamma': 10}
time: 19.091429281234742 	score: 0.605308753614884 	parameters: {'clf__C': 1000, 'clf__gamma': 0.1}
time: 17.595486545562743 	score: 0.6426686721997893 	parameters: {'clf__C': 1000, 'clf__gamma': 1}
time: 14.481508255004883 	score: 0.5122350651947486 	parameters: {'clf__C': 1000, 'clf__gamma': 10}


Weighted matricees are already numbers from 0 to 1, best accuracy is 0.63%, meaning this is currently the moste accurate method, maby an even higher C can give even better results?

In [55]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                 ('clf', SVC(gamma=1))])

params = {'clf__C': [100,1000,10000]}

grid = GridSearchCV(pipe, params, n_jobs=-1)
grid.fit(text, labels)

time = grid.cv_results_['mean_fit_time']
scores = grid.cv_results_['mean_test_score']
parameters = grid.cv_results_['params']

print('Best parameters:', grid.best_params_)
print('Best score:', grid.best_score_)
print('Size of model: none')
print('Size of stopword list: none')

for model in nb_nlp:
    for stop in no_stop:
        text = lemmatizer(text, nb_nlp[model], no_stop[stop])

        grid = GridSearchCV(pipe, parameters, n_jobs=-1)
        grid.fit(text, labels)

        print('Best parameters:', grid.best_params_)
        print('Best score:', grid.best_score_)
        print('Size of model:', model)
        print('Size of stopword list:', stop)

time: 18.146734714508057 	score: 0.6426686721997893 	parameters: {'clf__C': 100}
time: 18.08936176300049 	score: 0.6426686721997893 	parameters: {'clf__C': 1000}
time: 15.812409019470214 	score: 0.6426686721997893 	parameters: {'clf__C': 10000}


Larger normalization does not seem to improve the performance of SVC, but Smaler does not seem to decreese the performance either

In [58]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                 ('clf', SVC(gamma=1))])

params = {'clf__C': [1,5,7,10,100]}

grid = GridSearchCV(pipe, params, n_jobs=-1)
grid.fit(text, labels)

time = grid.cv_results_['mean_fit_time']
scores = grid.cv_results_['mean_test_score']
parameters = grid.cv_results_['params']

for i in range(0, len(time)):
    print(f'time: {time[i]} \tscore: {scores[i]} \tparameters: {parameters[i]}')

time: 14.907119083404542 	score: 0.6331644746070259 	parameters: {'clf__C': 1}
time: 16.471586418151855 	score: 0.6425596213362461 	parameters: {'clf__C': 5}
time: 18.462618112564087 	score: 0.6426686721997893 	parameters: {'clf__C': 7}
time: 18.148515748977662 	score: 0.6426686721997893 	parameters: {'clf__C': 10}
time: 16.538453245162962 	score: 0.6426686721997893 	parameters: {'clf__C': 100}


we see that a C between 5 and 7 is optimal, but we don't gain anything from findin the exact right C as we can se from the score of C=10

In [14]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                 ('scale', StandardScaler(with_mean=False)),
                 ('clf', MLPClassifier())])

params = {'clf__hidden_layer_sizes': [[10,10],[100,100],[1000,1000]]}

grid = GridSearchCV(pipe, params, n_jobs=-1)
grid.fit(text, labels)

scores = grid.cv_results_['mean_test_score']
parameters = grid.cv_results_['params']

print(f'best score:{grid.best_score_} parameters:{grid.best_params_}')

best score:0.5789823710057211 parameters:{'clf__hidden_layer_sizes': [1000, 1000]}
best score:0.5789823710057211 parameters:{'clf__hidden_layer_sizes': [1000, 1000]}
best score:0.5789823710057211 parameters:{'clf__hidden_layer_sizes': [1000, 1000]}


output:
```
best score:0.5789823710057211 parameters:{'clf__hidden_layer_sizes': [1000, 1000]}
best score:0.5789823710057211 parameters:{'clf__hidden_layer_sizes': [1000, 1000]}
best score:0.5789823710057211 parameters:{'clf__hidden_layer_sizes': [1000, 1000]}
```
we see that two hidden layers with 1000 items each is stil not overfitting

In [15]:
print(scores, parameters)

[0.55702614 0.57286514 0.57898237] [{'clf__hidden_layer_sizes': [10, 10]}, {'clf__hidden_layer_sizes': [100, 100]}, {'clf__hidden_layer_sizes': [1000, 1000]}]


In [ ]:
pipe = Pipeline([('vect', TfidfVectorizer()),
                 ('scale', StandardScaler(with_mean=False)),
                 ('clf', MLPClassifier())])

params = {'clf__hidden_layer_sizes': [[1000,1000], [10000,1000], [1000,1000,1000]]}

grid = GridSearchCV(pipe, params, n_jobs=-1)
grid.fit(text, labels)

scores = grid.cv_results_['mean_test_score']
parameters = grid.cv_results_['params']

print(f'best score:{grid.best_score_} parameters:{grid.best_params_}')

output:
```
[0.55702614 0.57286514 0.57898237] [{'clf__hidden_layer_sizes': [10, 10]}, {'clf__hidden_layer_sizes': [100, 100]}, {'clf__hidden_layer_sizes': [1000, 1000]}]
```
we see that the hidden layers are increasing the performance by 0.6% between two 100 item layers and two 1000 item layers, so we are probably close to overfitting

CountVectorizer, tokenizes, builds a vocabulary, and encodes the text, in other words, it seperates sentences into lists of words, removes norwegian stopwords (due to us defining norwegian stopwords), and builds dictionaries of the frequencie of each word across each sentence. the stoppwords used here are downloaded from the git: https://raw.githubusercontent.com/stopwords-iso/stopwords-no/master/stopwords-no.json

The transform function turns the dictionaries from "CountVectorizer" into vectors, and the "get_feature_names_out" function returns the words in the two vectors as a list in alphabetical order. we see that the vectors build on lemmatized sentences are smaler

still need to test neural network accuracy, and find the optimal classifier for binary sentiment analysis